# 🚀 Transformer

## Table of contents
0. [Parameters](#parameters)
1. [Load the Data](#load)
2. [Tokensise the Data](#tokenise)
3. [Create the Training Set](#create)
4. [Build the LSTM](#build)
5. [Train the LSTM](#train)

In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from pprint import pprint
import random
import re
import string

import tensorflow as tf
import tensorflow.keras as keras

## 0. Parameters <a name="parameters"></a>

In [66]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32

## 1. Load the data <a name="load"></a>

In [79]:
# Load the full dataset
with open('/app/data/wine-reviews/winemag-data-130k-v2.json') as json_data:
    wine_data = json.load(json_data)
    

In [80]:
wine_data[10]

{'points': '87',
 'title': 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)',
 'description': 'Soft, supple plum envelopes an oaky structure in this Cabernet, supported by 15% Merlot. Coffee and chocolate complete the picture, finishing strong at the end, resulting in a value-priced wine of attractive flavor and immediate accessibility.',
 'taster_name': 'Virginie Boone',
 'taster_twitter_handle': '@vboone',
 'price': 19,
 'designation': 'Mountain Cuvée',
 'variety': 'Cabernet Sauvignon',
 'region_1': 'Napa Valley',
 'region_2': 'Napa',
 'province': 'California',
 'country': 'US',
 'winery': 'Kirkland Signature'}

In [82]:
# Filter the dataset
filtered_data = ['wine review : ' + x['country'] + ' : ' + \
                 x['province']  + ' : ' + x['variety'] + 
                 ' : ' + x['description'] \
            for x in wine_data
              if x['country'] is not None
                 and x['province'] is not None
                 and x['variety'] is not None
                 and x['description'] is not None
             ]

In [83]:
# Count the recipes
n_wines = len(filtered_data)
print(f'{n_wines} recipes loaded')

129907 recipes loaded


In [84]:
example = filtered_data[25]
print(example)

wine review : US : California : Pinot Noir : Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site. Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture.


## 2. Tokenise the data

In [85]:
# Pad the punctuation, to treat them as separate 'words'  
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r' \1 ', s)
    s = re.sub(' +', ' ', s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]

In [86]:
# Display an example of a recipe
example_data = text_data[9]
example_data

"wine review : France : Alsace : Pinot Gris : This has great depth of flavor with its fresh apple and pear fruits and touch of spice . It ' s off dry while balanced with acidity and a crisp texture . Drink now . "

In [87]:
# Convert to a Tensorflow Dataset
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(BATCH_SIZE).shuffle(1000)

In [88]:
# Create a vectorisation layer
vectorize_layer = keras.layers.TextVectorization(
    standardize = 'lower',
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [89]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [92]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f'{i}: {word}')

0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


In [93]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  7  10   2  42   2 344   2  43 410   2  12  41 183 351   9 145  11  54
  56  69   5 115  70   5 108   9  49   4  15  18  21 227  58  80 109  11
  30   5   8  73  77   4  35  66   4   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0]


## 3. Create the Training Set

In [94]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

train_ds = text_ds.map(prepare_inputs)

## 2. Build the Transformer <a name="build"></a>

In [95]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.att = keras.layers.MultiHeadAttention(N_HEADS, EMBEDDING_DIM)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
        })
        return config

In [96]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size =vocab_size
        self.embed_dim = embed_dim
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config

In [97]:

def create_model():
    inputs = keras.layers.Input(shape=(MAX_LEN,), dtype=tf.int32)
    embedding_layer = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(EMBEDDING_DIM, N_HEADS, FEED_FORWARD_DIM)
    x = transformer_block(x)
    outputs = keras.layers.Dense(VOCAB_SIZE, activation = 'softmax')(x)
    model = keras.Model(inputs=inputs, outputs=[outputs, x])
    loss_fn = keras.losses.SparseCategoricalCrossentropy()
    model.compile(
        "adam", loss=[loss_fn, None],
    )  # No loss and optimization based on word embeddings from transformer block
    return model

model = create_model()

In [98]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 80)]              0         
                                                                 
 token_and_position_embeddin  (None, 80, 256)          2580480   
 g_2 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_2 (Transf  (None, 80, 256)          658688    
 ormerBlock)                                                     
                                                                 
 dense_8 (Dense)             (None, 80, 10000)         2570000   
                                                                 
Total params: 5,809,168
Trainable params: 5,809,168
Non-trainable params: 0
_________________________________________________

In [99]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    model = keras.models.load_model('./models/model', compile=False)

## 3. Train the LSTM <a name="train"></a>

In [100]:
# Create a TextGenerator checkpoint
class TextGenerator(keras.callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs 
    
    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            pad_len = MAX_LEN - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:MAX_LEN]
                sample_index = MAX_LEN - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y, _ = self.model.predict(x, verbose = 0)
            sample_token, probs = self.sample_from(y[0][sample_index], temperature)
            info.append({'prompt': start_prompt , 'word_probs': probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + ' ' + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info
        
    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens = 100, temperature = 1.0)
        

In [101]:
# Create a model save checkpoint
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [102]:
model.fit(
    train_ds, 
    epochs=25, 
    # steps_per_epoch = 3,
    callbacks = [model_checkpoint_callback, tensorboard_callback, text_generator]
)

Epoch 1/25
4060/4060 [==============================] - ETA: 0s - loss: 2.2502 - dense_8_loss: 2.2502
generated text:
wine review 
 : italy : northeastern italy : pinot grigio : you ' ll also that you can taste the lightest cuts of aromatic interest and lemon grass . there ' s a touch of tomato leaf in this rich wine , but the wine is steady , polished and straight proportioned . 

4060/4060 [==============================] - 115s 28ms/step - loss: 2.2502 - dense_8_loss: 2.2502
Epoch 2/25
2001/4060 [=============>................] - ETA: 56s - loss: 1.9812 - dense_8_loss: 1.9812

KeyboardInterrupt: 

In [103]:
# Save the final model
model.save("./models/model")

2022-07-11 22:50:58.911330: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./models/model/assets


INFO:tensorflow:Assets written to: ./models/model/assets


# 3. Generate text using the Transformer

In [104]:
def print_probs(info, vocab, top_k = 5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i['word_probs']
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f'{vocab[i]}:   \t{np.round(100*p,2)}%') 
        print('--------\n')

In [122]:
info = text_generator.generate("wine review : us"
                               , max_tokens = 20, temperature = 1.0)


generated text:
wine review : us : california : merlot : sweetly sweet , like leather and tobacco leaves in the glass



In [123]:
print_probs(info, vocab)


PROMPT: wine review : us
::   	99.99%
-:   	0.0%
.:   	0.0%
and:   	0.0%
grosso:   	0.0%
--------


PROMPT: wine review : us :
california:   	79.4%
washington:   	7.43%
oregon:   	7.04%
new:   	4.1%
virginia:   	1.12%
--------


PROMPT: wine review : us : california
::   	99.99%
other:   	0.0%
valley:   	0.0%
grosso:   	0.0%
.:   	0.0%
--------


PROMPT: wine review : us : california :
pinot:   	39.49%
chardonnay:   	13.92%
cabernet:   	11.34%
red:   	5.8%
syrah:   	5.09%
--------


PROMPT: wine review : us : california : merlot
::   	99.51%
-:   	0.4%
,:   	0.03%
blanc:   	0.02%
and:   	0.01%
--------


PROMPT: wine review : us : california : merlot :
this:   	28.81%
a:   	10.89%
the:   	3.52%
there:   	2.04%
very:   	1.96%
--------


PROMPT: wine review : us : california : merlot : sweetly
ripe:   	41.13%
fruity:   	8.35%
caramelized:   	6.76%
candied:   	2.96%
delicious:   	2.93%
--------


PROMPT: wine review : us : california : merlot : sweetly sweet
,:   	38.53%
and:   	26.66%
i

In [ ]:
info = text_generator.generate("recipe for roasted vegetables | chop 1 /", max_tokens = 10, temperature = 0.2)

In [ ]:
print_probs(info, vocab)

In [ ]:
info = text_generator.generate("recipe for chocolate ice cream |", max_tokens = 7, temperature = 1.0)
print_probs(info, vocab)

In [ ]:
info = text_generator.generate("recipe for chocolate ice cream |", max_tokens = 7, temperature = 0.2)
print_probs(info, vocab)